In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, div, dx, ds, sqrt
import icepack, icepack.models, icepack.plot

# Synthetic ice stream

In this demo, we'll simulate the evolution of an ice stream with both grounded and floating parts.
Ice streams are substantially more complicated than a floating ice shelf.
First, there are more fields.
For floating ice shelves, we only had to consider the velocity, thickness, and fluidity, but for ice streams there's also the surface elevation and bed friction.
Second, the position of the grounded line -- the contour across which the glacier is thin enough to go afloat -- can migrate in time.

The diagnostic equation for an ice stream is similar to that of a floating ice shelf, but for the addition of a term for basal friction:

$$\nabla\cdot hM - C|u|^{1/m - 1}u - \rho gh\nabla s = 0,$$

where $m$ is the *sliding exponent* and $C$ is the *sliding coefficient*.

The glacier state we'll start out with is grounded throughout the entire domain, and during the simulation it will thin out and go afloat part way through.
Accurately predicting grounding line migration is a major problem in glacier flow modeling.

### Geometry and input data

We'll use an elongated fjord-like geometry, 12 km wide and 50 km from the inflow boundary to the ice front.
The bedrock slopes down from 200 m above sea level at the inflow boundary to -400 m at the terminus; the glacier becomes marine one third of the way into the domain.

In [ ]:
Lx, Ly = 50e3, 12e3
nx, ny = 32, 32
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly)

degree = 2
Q = firedrake.FunctionSpace(mesh, 'CG', degree)
V = firedrake.VectorFunctionSpace(mesh, 'CG', degree)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

b_in, b_out = 200, -400
b = firedrake.interpolate(b_in - (b_in - b_out) * x / Lx, Q)

s_in, s_out = 1000, 50
s0 = firedrake.interpolate(s_in - (s_in - s_out) * x / Lx, Q)

h0 = firedrake.interpolate(s0 - b, Q)

As a sanity check, we'll evaluate the driving stress

$$\tau_D = -\rho_Igh\nabla s$$

at the inflow boundary of the ice stream to make sure the value isn't absurd.

In [ ]:
from icepack.constants import rho_ice as ρ_I, rho_water as ρ_W, gravity as g
h_in = s_in - b_in
δs_δx = (s_out - s_in) / Lx
τ_D = -ρ_I * g * (s_in - b_in) * (s_out - s_in) / Lx
print("{} kPa".format(1000 * -ρ_I * g * h_in * δs_δx))

This stress is on the moderate side for Greenland, high for Antarctica.

Next we have to come up with some rough guess for the initial velocity of the glacier.
Unlike for ice shelves, we can't write down an exact solution for the velocity.
Our best hope is to come up with something vaguely reasonable and tune it by trial and error until we get something that isn't ridiculous.

In [ ]:
u_in, u_out = 500, 8500
velocity_x = u_in + (u_out - u_in) * (x/Lx)**2
u0 = firedrake.interpolate(firedrake.as_vector((velocity_x, 0)), V)

In [ ]:
temp = 255.0
A0 = icepack.rate_factor(temp)
A = firedrake.interpolate(firedrake.Constant(A0), Q)

Now to pick a friction coefficient.
We'll use a value of the friction that exactly balances the driving stress at the inflow boundary.
Where the glacier goes marine, we'll scale the friction down acoording to the height above flotation.
This is a very rough approximation.
The point is to scale the friction down continuously as the glacier approaches flotation.
If we didn't take care to ramp the friction down and instead just set it to 0 where the ice is floating, we'd get a shock to the system as the bed friction "turns off" across the grounding line.

In [ ]:
from icepack.constants import weertman_sliding_law as m
friction = τ_D / u_in**(1/m)

p_W = ρ_W * g * firedrake.max_value(0, h0 - s0)
p_I = ρ_I * g * h0
ϕ = 1 - p_W / p_I
C = firedrake.interpolate(friction * ϕ, Q)

The last thing we need to do before jumping off into the actual simulation is to compute the initial velocity field.

In [ ]:
ice_stream = icepack.models.IceStream()
opts = {'dirichlet_ids': [1], 'side_wall_ids': [3, 4], 'tol': 1e-6}
u0 = ice_stream.diagnostic_solve(u0=u0, h=h0, s=s0, A=A, C=C, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u0, 25, axes=axes)
fig.colorbar(contours, ax=axes, fraction=0.012, pad=0.04)
plt.show(fig)

Below we show some plots of the basal shear stress and the mass flux.
Note how the basal shear stress gets closer to 0 near the terminus because of how we reduced the friction.

In [ ]:
τ_b = firedrake.interpolate(-1000 * C * sqrt(inner(u0, u0))**(1/m - 1)*u0, V)
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(τ_b, 25, axes=axes)
fig.colorbar(contours, label='kPa', fraction=0.012, pad=0.04)
plt.show(fig)

There's a huge mass balance anomaly near the ice front, an artifact of our choice of initial conditions.
As the simulation progresses, this initial transient should be propagated out of the domain.

In [ ]:
f = firedrake.interpolate(-div(h0 * u0), Q)
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(f, 25, cmap='RdBu', vmin=-144, vmax=144, axes=axes)
fig.colorbar(contours, label='meters/year', fraction=0.012, pad=0.04)
plt.show(fig)

### Modeling

Now the good part -- taking our initial glacier state and projecting it forward until it reaches a steady state.
We'll use a timestep of 2 months and run the simulation forward for 32 model years.
This is about enough time to propagate out the initial transients and arrive at a state of mass and momentum balance.
Additionally, we choose an accumulation rate that roughly balances out the ice flux at the inflow boundary, and gives a vaguely reasonable melt rate towards the front.

In [ ]:
num_years = 32
timesteps_per_year = 6

δt = 1.0/timesteps_per_year
num_timesteps = num_years * timesteps_per_year

a = 6 - 12 * x / Lx
h = h0.copy(deepcopy=True)
u = u0.copy(deepcopy=True)

for k in range(num_timesteps + 1):
    h = ice_stream.prognostic_solve(δt, h0=h, a=a, u=u, h_inflow=h0)
    s = ice_stream.compute_surface(h=h, b=b)
    ϕ = 1 - ρ_W/ρ_I * firedrake.max_value(0, h - s) / h
    C = firedrake.interpolate(friction * ϕ, Q)
    u = ice_stream.diagnostic_solve(u0=u, h=h, s=s, A=A, C=C, **opts)
    print('.' if k % timesteps_per_year == 0 else '', end='')

Now some plots of the results:

In [ ]:
fig, axes = icepack.plot.subplots()
contours =  icepack.plot.tricontourf(h, 25, axes=axes)
fig.colorbar(contours, label='meters', fraction=0.012, pad=0.04)
plt.show(fig)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, 25, axes=axes)
fig.colorbar(contours, label='meters/year', fraction=0.012, pad=0.04)
plt.show(fig)

We can see by plotting the mass balance that the system is much closer to equilibrium than it was at the start.

In [ ]:
f = firedrake.interpolate(a - div(h * u), Q)
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(f, 25, cmap='RdBu', vmin=-1.2, vmax=1.2, axes=axes)
fig.colorbar(contours, label='meters/year', fraction=0.012, pad=0.04)
plt.show(fig)

Finally, we can draw some transects through the domain to see what the surface, bed, and ice base elevation look like in the $x, z$-plane.

In [ ]:
import numpy as np
xs = np.array([(Lx * k / (2 * nx), 0) for k in range(2 * nx + 1)])

ss = np.array(s.at(xs, tolerance=1e-10))
hs = np.array(h.at(xs, tolerance=1e-10))
bs = np.array(b.at(xs, tolerance=1e-10))

fig, ax = plt.subplots()
ax.plot(xs[:, 0], bs, color='black')
ax.plot(xs[:, 0], ss - hs, color='blue')
ax.plot(xs[:, 0], ss, color='blue')
plt.show(fig)

The glacier exhibits the typical drop in thickness gradient as it crosses the grounding line.

In [ ]:
fig, ax = plt.subplots()
ax.plot(xs[:, 0], hs)
plt.show(fig)

### Conclusion

In this demo we've shown how to simulate the evolution of a grounded ice stream.
There are more fields to keep track of than for a floating ice shelf; in addition to updating the ice thickness and velocity in each timestep, we must also update the surface elevation and the friction coefficient to account for possible ungrounding.
Nonetheless, the basic principles are still essentially the same.

Coming up with physically plausible synthetic test cases is difficult.
The variables that we can control are the bed geometry, the accumulation rate, and the ice thickness and velocity at inflow.
To a certain extent, we can pick the friction coefficient, but this can change in ways that are difficult to predict as parts of the glacier begin to float.
Our initial state was far out of mass balance and we let it evolve towards equilibrium, but we don't know ahead of time what surface slope or stretching rate the glacier will settle on in the end.

The eventual steady state could be physically implausible for reasons that weren't obvious from the start.
For example, if the ice strain rate increases beyond what we expect after a few years of model time, the mass flux at the inflow boundary could be much larger than what the influx and accumulation rate can supply, leading to an unphysically sharp drop in the surface.

My point is, if you try to come up with your own synthetic test cases, expect lots of trial and error.

The bed elevation we used sloped down towards the ocean; had we instead used a bed elevation that sloped down going inland, the configuration would be unstable due to marine ice sheet instability.
An interesting variations on this demo could be to use a bed geometry with several stable pinning points, interspersed with unstable retrograde slopes.